In [102]:
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
#from feature_engineering import add_retning
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
sys.path.append('../data')
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler


from display_data import import_data

In [103]:
def add_radius(df):
    #adds radius column to dataframe
    df['radius'] = np.sqrt((df['latitude']-55.75)**2 + (df['longitude']-37.55)**2)
    return df


def fix_geo_data(data_test):
    """should just be called on test data"""
    #fix all radius issues
    data_test._set_value(23,'longitude',37.473761)
    data_test._set_value(23,'latitude',55.560891)
    data_test._set_value(90,'longitude',37.473761)
    data_test._set_value(90,'latitude',55.560891)

    data_test._set_value(2511,'longitude',37.478055)
    data_test._set_value(2511,'latitude',55.544046)
    data_test._set_value(5090,'longitude',37.478055)
    data_test._set_value(5090,'latitude',55.544046)
    data_test._set_value(6959,'longitude',37.478055)
    data_test._set_value(6959,'latitude',55.544046)
    data_test._set_value(8596,'longitude',37.478055)
    data_test._set_value(8596,'latitude',55.544046)

    data_test._set_value(4719,'longitude',37.385493)
    data_test._set_value(4719,'latitude',55.853117)

    data_test._set_value(9547,'longitude',37.384711)
    data_test._set_value(9547,'latitude',55.853511)

    data_test._set_value(2529,'longitude',37.464994)
    data_test._set_value(2529,'latitude',55.627666)

    data_test = add_radius(data_test)

    return data_test

def add_high_up(df):
    """add exponetital function to determine how high up a building is"""
    high_up = df.floor/df.stories
    high_up_exp = np.exp(high_up) - 1
    euler = np.exp(1)

    df['high_up'] = high_up_exp
    df['high_up'].where(df['high_up'] > euler, euler)


    return df


In [104]:
from sklearn.model_selection import train_test_split

#We dont want our model to care about the id of the house or the seller
#In my first run, i will replace missing values with the mean value

data, data_test = import_data()
Y = data.price
data_RR = pd.DataFrame()
data_RR['area_total'] = data['area_total']
#data = fix_geo_data(data)
data = add_radius(data)
data_RR['radius'] = data['radius']

data = add_high_up(data)
#data_RR['high_up'] = data['high_up'] # Gjør dårligere


ceiling_mean = data['ceiling'].mean()
data['ceiling'] = data['ceiling'].replace(np.NaN, ceiling_mean)
#data_RR['ceiling'] = data['ceiling'] ### GJør ikke bedre

scaler = MinMaxScaler() # mapper alt til mellom 0 og 1, default
data_RR = scaler.fit_transform(data_RR)

y_plot = np.log(Y.values)/np.log(15)
#r_log_plot = np.log(data_RR['radius'].values*100)

#plt.plot(r_log_plot,y_plot)
#plt.show()
#plt.plot(data_RR['area_total'].values,y_plot)
#plt.show
#print(data_RR)
# add ceiling

In [105]:
X_train_RR, X_test_RR, y_train_RR, y_test_RR = train_test_split(data_RR, Y, test_size=0.2, random_state=42)


RR = linear_model.RANSACRegressor()
y_train_RR = np.log(y_train_RR)/np.log(15)

RR.fit(X_train_RR, y_train_RR)
prediction_RR = RR.predict(X_test_RR)
prediction_RR = 15**prediction_RR

def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

rmsle = root_mean_squared_log_error(y_test_RR,prediction_RR)
print("first run", rmsle)


first run 0.45870611432567193


uten scaling: 0.4749
med scaling: 0.45870611432567193
